In [1]:
import tensorflow
from tensorflow import keras
import numpy as np
import pandas as pd
import PIL
from PIL import Image
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split 
import cv2

C:\Users\Samno\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Samno\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Samno\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Samno\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [16]:
data=[]
labels=[]
path=r"D:\Traffic sign recognition"
classes=43
for i in range(classes):
    pathes=os.path.join(path,'train',str(i))
    images=os.listdir(pathes)
    for a in images:
        try:
            img=cv2.imread(pathes+'\\'+a)
            img=cv2.resize(img,(30,30))
            img=np.array(img)
            data.append(img)
            labels.append(i)
        except:
            print("Error processing image")
data=np.array(data)
labels=np.array(labels)

In [17]:
print(data.shape)

(39209, 30, 30, 3)


In [18]:
xtrain,xtest,ytrain,ytest=train_test_split(data,labels,test_size=0.2)
print(xtrain.shape,ytrain.shape)
ytrain=to_categorical(ytrain)
ytest=to_categorical(ytest)

(31367, 30, 30, 3) (31367,)


In [21]:
model=Sequential()
model.add(Conv2D(32,(5,5),activation='relu',input_shape=xtrain.shape[1:]))
model.add(Conv2D(32,(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43,activation="softmax"))

model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

In [22]:
history=model.fit(xtrain,ytrain,epochs=15,batch_size=64,validation_data=(xtest,ytest))

Train on 31367 samples, validate on 7842 samples
Epoch 1/15
31367/31367 [==============================] - 126s 4ms/sample - loss: 1.7821 - acc: 0.5647 - val_loss: 0.2488 - val_acc: 0.9445
Epoch 2/15
31367/31367 [==============================] - 126s 4ms/sample - loss: 0.4359 - acc: 0.8828 - val_loss: 0.1203 - val_acc: 0.9685
Epoch 3/15
31367/31367 [==============================] - 126s 4ms/sample - loss: 0.2768 - acc: 0.9254 - val_loss: 0.0924 - val_acc: 0.9796
Epoch 4/15
31367/31367 [==============================] - 126s 4ms/sample - loss: 0.2198 - acc: 0.9394 - val_loss: 0.0555 - val_acc: 0.9864
Epoch 5/15
31367/31367 [==============================] - 126s 4ms/sample - loss: 0.1865 - acc: 0.9493 - val_loss: 0.0571 - val_acc: 0.9871
Epoch 6/15
31367/31367 [==============================] - 126s 4ms/sample - loss: 0.1733 - acc: 0.9550 - val_loss: 0.0429 - val_acc: 0.9899
Epoch 7/15
31367/31367 [==============================] - 126s 4ms/sample - loss: 0.1470 - acc: 0.9608 - val_lo

In [28]:
from sklearn.metrics import accuracy_score
testdata=pd.read_csv(r"D:\Traffic sign recognition\Test.csv")
label=testdata["ClassId"].values
imgs=testdata["Path"].values
data=[]
for imge in imgs:
    image=cv2.imread("D:\Traffic sign recognition"+"\\" + imge)
    image=cv2.resize(image,(30,30))
    data.append(np.array(image))
    
xtest=np.array(data)
pred=model.predict_classes(xtest)
accuracy_score(pred,label)

0.9588281868566905

In [29]:
model.save('traffic_sign_classifier.h5')